---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [76]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [99]:
import re

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [40]:
def answer_one():
    t = df[(df['Gold'] == df['Gold'].max())].index[0]
    return t

In [41]:
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [63]:
def answer_two():
    diff = abs(df['Gold'] - df['Gold.1'])

    t = diff[(diff == diff.max())].index[0]
    
    return t

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    return "YOUR ANSWER HERE"

In [132]:
def answer_three():
    new_df = df[(df['Gold'] >0) & (df['Gold.1'] > 0)]

    m_list = list(map(lambda x: re.match("Gold[.]*[0-9]*",x), list(new_df.columns)))

    gold_cols = []
    for x in m_list:
        if x is not None:
            gold_cols.append(x.group(0))
        
    new_df['total_gold'] = new_df[gold_cols].sum(axis=1)


    rel_df = (new_df['Gold'] - new_df['Gold.1'])/(new_df['total_gold'])
    t = rel_df[rel_df == rel_df.max()].index[0]
    return t

In [133]:
answer_three()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [145]:
def answer_four():
    w_gold = 3
    w_silver = 2
    w_bronze = 1
    
    m_df = df.copy()
    m_df['Points'] = m_df['Gold.2'] * w_gold + m_df['Silver.2'] * w_silver + m_df['Bronze.2'] * w_bronze
    return m_df['Points']

In [146]:
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [77]:
import pandas as pd

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [79]:
#census_county = census_df.copy()

#census_county[census_county['SUMLEV'] == 50]

#census_county

In [56]:
#all_states = census_county['STATE'].unique()

#t = list(map(lambda x: (census_county['STATE'] == x).sum(), all_states))
#ind_max = all_states[t == max(t)][0]

#census_county['STNAME'][census_county['STATE'] == ind_max].unique()[0]

'Texas'

In [3]:
def answer_five():
    import pandas as pd
    census_df = pd.read_csv('census.csv')
    
    census_county = census_df.copy()

    census_county = census_county[census_county['SUMLEV'] == 50]

    all_states = census_county['STATE'].unique()

    t = list(map(lambda x: (census_county['STATE'] == x).sum(), all_states))
    ind_max = all_states[t == max(t)][0]

    big_state = census_county['STNAME'][census_county['STATE'] == ind_max].unique()[0]
    return big_state

In [4]:
answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

##### I need to set multi-level indices here

In [91]:
import pandas as pd
census_df = pd.read_csv('census.csv')
    
census_county = census_df.copy()

census_county = census_county[census_county['SUMLEV'] == 50]

In [1]:
#census_county = census_county.set_index(['STNAME', 'CTYNAME'])
#census_county.groupby([census_county.index.get_level_values(0), census_county.index.get_level_values(1)])

In [3]:
census_county1 = census_county.set_index(['STNAME', 'CTYNAME'])
#df_tmp = census_county.groupby(['STNAME', 'CTYNAME']).agg({'CENSUS2010POP':sum})

## group on state level, we apply a function to the whole series which includes CTYNAME and Population
top_3_counties = census_county1['CENSUS2010POP'].groupby(level=['STNAME'], group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(3))
## group on state and county level, we apply a function to each value in the Population column 
#census_county1['CENSUS2010POP'].groupby(level=['STNAME', 'CTYNAME'], group_keys=True).apply(lambda x: x.values)

top_3_counties

#t1 = df_tmp['CENSUS2010POP'].groupby(level=0, group_keys=False)


#t1.nlargest(3)
#df_tmp1 = census_county.sort(['CENSUS2010POP'],ascending=False).groupby(['STNAME', 'CTYNAME'])

#df_tmp1.sum()
# another way to do the above
#df_tmp2 = census_county.groupby(['STNAME','CTYNAME']) # pretty much the same as multi-level indexing
#df_tmp2.sum()
# end another way
## sort index
#df_tmp.sortlevel(ascending = True)
## end

#df_tmp.sort_values(by=['STNAME','CTYNAME','CENSUS2010POP'], inplace = False)

#t = df_tmp.sort([('CENSUS2010POP')], ascending=False)

#t

#print(df_tmp)
#df_tmp['CENSUS2010POP']
#df_tmp.index.get_level_values(0)

STNAME                CTYNAME                     
Alabama               Jefferson County                 658466
                      Mobile County                    412992
                      Madison County                   334811
Alaska                Anchorage Municipality           291826
                      Fairbanks North Star Borough      97581
                      Matanuska-Susitna Borough         88995
Arizona               Maricopa County                 3817117
                      Pima County                      980263
                      Pinal County                     375770
Arkansas              Pulaski County                   382748
                      Benton County                    221339
                      Washington County                203065
California            Los Angeles County              9818605
                      San Diego County                3095313
                      Orange County                   3010232
Colorado           

In [10]:
ans = top_3_counties.groupby(level=['STNAME']).apply(lambda x: sum(x)).sort_values(ascending=False)

In [20]:
ans[:3].index.tolist()

['California', 'Texas', 'Illinois']

In [21]:
def answer_six():
    import pandas as pd
    census_df = pd.read_csv('census.csv')
    
    census_county = census_df.copy()

    census_county = census_county[census_county['SUMLEV'] == 50]
    
    census_county1 = census_county.set_index(['STNAME', 'CTYNAME'])

    ## group on state level, we apply a function to the whole series which includes CTYNAME and Population
    top_3_counties = census_county1['CENSUS2010POP'].groupby(level=['STNAME'], group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(3))
    
    ans = top_3_counties.groupby(level=['STNAME']).apply(lambda x: sum(x)).sort_values(ascending=False)
    return ans[:3].index.tolist()

In [22]:
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [84]:
#census_county_filtered = census_county.loc[:, 'SUMLEV':'POPESTIMATE2015']

In [92]:
census_county['LAC'] = census_county.loc[:,'POPESTIMATE2010':'POPESTIMATE2015'].apply(lambda x: max(x)-min(x), axis = 1)

In [126]:
census_county['LAC'].idxmax()

2667

In [128]:
len(census_county['LAC'])
census_county[census_county.index == 2667]
#census_county['LAC']

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,LAC
2667,50,3,7,48,201,Texas,Harris County,4092459,4093076,4108187,...,3.518819,4.820913,5.243613,3.794291,7.154417,8.915715,10.683269,11.850629,10.303814,429841


In [145]:
census_county.loc[2667]

SUMLEV                              50
REGION                               3
DIVISION                             7
STATE                               48
COUNTY                             201
STNAME                           Texas
CTYNAME                  Harris County
CENSUS2010POP                  4092459
ESTIMATESBASE2010              4093076
POPESTIMATE2010                4108187
POPESTIMATE2011                4181238
POPESTIMATE2012                4262504
POPESTIMATE2013                4352462
POPESTIMATE2014                4447577
POPESTIMATE2015                4538028
NPOPCHG_2010                     15111
NPOPCHG_2011                     73051
NPOPCHG_2012                     81266
NPOPCHG_2013                     89958
NPOPCHG_2014                     95115
NPOPCHG_2015                     90451
BIRTHS2010                       16103
BIRTHS2011                       66939
BIRTHS2012                       66391
BIRTHS2013                       67638
BIRTHS2014               

In [123]:
census_county.iloc[census_county['LAC'].idxmax()]

SUMLEV                            50
REGION                             3
DIVISION                           7
STATE                             48
COUNTY                           289
STNAME                         Texas
CTYNAME                  Leon County
CENSUS2010POP                  16801
ESTIMATESBASE2010              16801
POPESTIMATE2010                16745
POPESTIMATE2011                16830
POPESTIMATE2012                16738
POPESTIMATE2013                16652
POPESTIMATE2014                16791
POPESTIMATE2015                17086
NPOPCHG_2010                     -56
NPOPCHG_2011                      85
NPOPCHG_2012                     -92
NPOPCHG_2013                     -86
NPOPCHG_2014                     139
NPOPCHG_2015                     295
BIRTHS2010                        42
BIRTHS2011                       203
BIRTHS2012                       225
BIRTHS2013                       204
BIRTHS2014                       208
BIRTHS2015                       202
D

In [164]:
def answer_seven():
    import pandas as pd
    census_df = pd.read_csv('census.csv')
    census_county = census_df.copy()
    census_county = census_county[census_county['SUMLEV'] == 50]
    census_county['LAC'] = census_county.loc[:,'POPESTIMATE2010':'POPESTIMATE2015'].apply(lambda x: abs(max(x)-min(x)), axis = 1)
    ans = census_county.loc[census_county['LAC'].idxmax()]
    return ans['CTYNAME']


In [165]:
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [177]:
import re

In [175]:
census_df = pd.read_csv('census.csv')
census_county = census_df.copy()
census_county = census_county[census_county['SUMLEV'] == 50]
county_reg = census_county[(census_county['REGION'] == 1) | (census_county['REGION'] == 2)]

In [195]:
def matchme(x):
    
    match = re.search("^Washington.*",x)
    if not match == None:
        
        return True
    else:
        return False
    
county_reg[county_reg['CTYNAME'].apply(matchme)]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
703,50,2,3,17,189,Illinois,Washington County,14716,14716,14701,...,-8.057633,2.192006,-12.192182,-2.775850,-10.605638,-8.125918,2.123506,-12.192182,-2.775850,-10.605638
799,50,2,3,18,175,Indiana,Washington County,28262,28262,28284,...,-3.683894,-9.517699,-5.310179,0.717901,-5.454774,-3.258829,-9.161231,-4.879624,1.148641,-5.024134
896,50,2,4,19,183,Iowa,Washington County,21704,21704,21697,...,5.743693,2.468684,-0.364282,0.272171,1.849596,5.881542,2.560117,-0.273212,0.408256,1.984933
1005,50,2,4,20,201,Kansas,Washington County,5799,5799,5782,...,10.504563,-18.164519,-18.821460,-5.501819,-6.948156,12.054417,-16.780555,-17.238347,-3.549561,-4.988420
1211,50,1,1,23,29,Maine,Washington County,32856,32856,32824,...,0.213626,-3.280699,-4.262285,-9.111049,-4.284274,0.366217,-2.790127,-3.829879,-8.643015,-3.811744
1419,50,2,4,27,163,Minnesota,Washington County,238136,238134,238995,...,2.555118,3.260957,2.285952,3.072543,0.678755,4.178076,4.907905,4.009584,5.040262,2.615204
1618,50,2,4,29,221,Missouri,Washington County,25195,25196,25177,...,-5.130755,-2.389867,0.079577,-1.592420,-11.511311,-5.011435,-2.389867,0.079577,-1.592420,-11.511311
1770,50,2,4,31,177,Nebraska,Washington County,20234,20234,20275,...,-1.874784,-1.331459,-4.492607,-0.889284,-3.998716,-1.825448,-1.282146,-4.393868,-0.790475,-3.899983
1918,50,1,2,36,115,New York,Washington County,63216,63216,63292,...,-3.719090,-1.776030,-4.165839,-4.954213,-2.342897,-3.418398,-1.522311,-3.879636,-4.634587,-2.021953
2162,50,2,3,39,167,Ohio,Washington County,61778,61778,61708,...,0.129798,0.780050,-0.455611,-2.137467,0.016355,0.227147,1.023816,-0.162718,-1.778503,0.376173


In [200]:
county_w = county_reg[county_reg['CTYNAME'].apply(lambda x: bool(re.search("^Washington.*", x)))]

In [210]:
ans = county_w[county_w['POPESTIMATE2015'] > county_w['POPESTIMATE2014']][['STNAME','CTYNAME']]

In [212]:
ans = ans.sort_index(ascending=True)

In [213]:
def answer_eight():
    import pandas as pd
    import re
    census_df = pd.read_csv('census.csv')
    census_county = census_df.copy()
    census_county = census_county[census_county['SUMLEV'] == 50]
    
    county_reg = census_county[(census_county['REGION'] == 1) | (census_county['REGION'] == 2)]
    county_w = county_reg[county_reg['CTYNAME'].apply(lambda x: bool(re.search("^Washington.*", x)))]
    ans = county_w[county_w['POPESTIMATE2015'] > county_w['POPESTIMATE2014']][['STNAME','CTYNAME']]
    ans = ans.sort_index(ascending=True)
    
    return ans

In [214]:
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
